<a href="https://colab.research.google.com/github/boyoung21/ESAA/blob/main/1013_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천시스템

현재 전자상거래 기업부터 OTT 플랫폼까지 다양한 분야에 활용되는 시스템

- 사용자들이 자각하지 못한 취향을 발견하고 그에 맞는 콘텐츠를 추천하므로써 고객 충성도 확보
- 온라인 사이트의 공간적 제약 부재로 더 많은 상품을 볼 수 있게 되면서 사용자들은 오히려 선택에 어려움을 겪음. 이럴 때 사용자의 데이터에 기반하여 추천 시스템이 어울릴 만한 것들을 추천하기도 함.

# 추천 시스템의 유형

## 콘텐츠 기반 필터링

사용자가 특정 아이템을 매우 선호하는 경우에, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식. 사용자가 특정 영화에 매긴 평점을 바탕으로 높은 점수를 기록한 영화들의 키워드를 분석하여 키워드가 유사한 다른 작품을 추천하는 방식 등.

## 협업 필터링

사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식을 기반으로 추천 수행. 콘텐츠 기반은 콘텐츠에 중점을 둔다면, 이것은 사용자의 행동 양식에 초점을 둠.

목표: 사용자가 아직 평가하지 않은 아이템을 예측 /  평가하는 것.

1. 최근접 이웃 협업 필터링

- 사용자 기반: 특정 사용자와 다른 사용자 간의 유사도에 기반하여 추천. 행 - 사용자, 열 - 아이템. 즉, 비슷한 고객이 뭘 구매했는지가 관건.
- 아이템 기반: 평가 척도가 유사한 아이템을 추천하는 것. 행 - 아이템, 열 - 사용자. 즉, 특정 물건을 샀을 때 그것을 구매한 다른 고객들의 구매 목록을 필요로 함.

일반적으로 아이템 기반 협업 필터링 정확도가 더 높음. 사용자 기반은 사용자가 선호에 대한 데이터를 남기지 않는 경우도 많고, 대중성을 띤 아이템일 경우 애매하기 때문.

2. 잠재요인 협업 필터링

사용자-아이템 평점 매트릭스 속에 숨어 있는 잠재요인을 추출해 추천 예측을 하는 방식. 이때 잠재요인은 명확하게 정의되지 않음.

- 대규모 다차원 행렬을 차원 감소 기법으로 분해하는 과정에서 잠재요인이 추출되며, 이 과정을 행렬 분해라 함.

- 사용자-아이템 행렬 데이터를 사용자-잠재요인 / 잠재요인-아이템 행렬로 분해. 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어서 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점까지 생성.

- 해당 기법에서는 주로 SVD(singular value decomposition 사용)
> SVD는 nan 값이 없는 행렬에만 사용 가능하기 때문에, 결측치가 존재한다면 SGD나 ALS 방식 사용해서 수행해야 함.

# 확률적 경사하강법을 이용한 행렬 분해

P와 Q.T 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추하는 것

1. P와 Q를 임의의 값을 가진 행렬로 설정
2. P와 Q.T값을 곱해 예측 R 행렬을 계산하고 이를 실제 R 행렬을 이용해 오류 값 계산
3. 2에서 오류가 최소화되도록 P와 Q 행렬을 적절한 값으로 업데이트
4. 괜찮다 싶을 때까지 2, 3을 반복해서 P와 Q를 업데이트, 근사화

In [3]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 k는 3으로 설정
R = np.array([[4, np.nan, np.nan, 2 ,np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])

num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [4]:
# R 행렬과 예측 행렬의 오차를 구하는 get_rmse() 함수 생성
# 실제 R 행렬의 널이 아닌 행렬 값의 위치 인덱스를 추출
# 이 인덱스에 있는 실제 R 행렬 값과 분해된 P, Q를 이용해 다시 조합된 예측 행렬 값의 RMSE 값 반환

from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [5]:
# SGD 기반 행렬 분해 수행

# R > 0인 행 위치, 열 위치, 값을 non-zeros 리스트에 저장.
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000 # SGD를 반복해서 업데이트할 횟수
learning_rate = 0.01 # SGD 학습률
r_lambda = 0.01 # L2 계수

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0: # 50회 반복할 때마다 오류 값을 출력
        print("### iteration step : ", step, " rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [6]:
# 분해된 P와 Q 함수를 P*Q.T로 예측 행렬을 만들어 출력

pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
